In [1]:
from knowledge_graph_maker import GraphMaker, Ontology, GroqClient, OpenAIClient
from knowledge_graph_maker import Document

In [2]:
from lotr_wikipedia_summary import lord_of_the_rings_wikipedia_summary as example_text_list
len(example_text_list)

2

In [3]:
ontology = Ontology(
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)

In [4]:
oai_model="gpt-3.5-turbo-0125"

## Use Groq
# llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
## OR Use OpenAI
llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)

In [5]:
import datetime
current_time = str(datetime.datetime.now())


graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary


docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    example_text_list
)

In [6]:
graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=0 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  gpt-3.5-turbo-0125


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-3.5-turbo-0125


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER LOG - 2024-05-23 05:40:38 - INFO 
Document: 1


▶︎ GRAPH MAKER VERBOSE - 2024-05-23 05:40:38 - INFO 
Using Ontology:
labels=[{'Person': 'Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun'}, {'Object': "Do not add the definite article 'the' in the object name"}, {'Event': 'Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc.'}, 'Place', 'Document', 'Organisation', 'Action', {'Miscellaneous': 'Any important concept can not be categorised with any other given label'}] relationships=['Relation between any pair of Entities']



Using Model:  gpt-3.5-turbo-0125


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-23 05:40:50 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Person",
            "name": "Bilbo Baggins"
        },
        "node_2": {
            "label": "Person",
            "name": "Frodo"
        },
        "relationship": "Bilbo Baggins celebrates his birthday and leaves the Ring to Frodo, his heir."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Gandalf"
        },
        "node_2": {
            "label": "Person",
            "name": "Frodo"
        },
        "relationship": "Gandalf suspects the Ring is a Ring of Power and counsels Frodo to take it away from the Shire."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Gandalf"
        },
        "node_2": {
            "label": "Person",
            "name": "Sauron"
        },
        "relationship": "Gan

Using Model:  gpt-3.5-turbo-0125


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-23 05:40:59 - INFO 
LLM Response:
```
[
    {
        "node_1": {
            "label": "Person",
            "name": "Merry"
        },
        "node_2": {
            "label": "Person",
            "name": "Pippin"
        },
        "relationship": "Merry and Pippin are trapped by the malign Old Man Willow."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Merry"
        },
        "node_2": {
            "label": "Person",
            "name": "Pippin"
        },
        "relationship": "Merry and Pippin are rescued by Tom Bombadil."
    },
    {
        "node_1": {
            "label": "Person",
            "name": "Frodo"
        },
        "node_2": {
            "label": "Person",
            "name": "Tom Bombadil"
        },
        "relationship": "Frodo calls Tom Bombadil to free them from the barrow-wight."
    },
   

Total number of Edges 20


In [7]:
for edge in graph:
    print(edge.model_dump(exclude=['metadata']), "\n\n")

{'node_1': {'label': 'Person', 'name': 'Bilbo Baggins'}, 'node_2': {'label': 'Person', 'name': 'Frodo'}, 'relationship': 'Bilbo Baggins celebrates his birthday and leaves the Ring to Frodo, his heir.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Gandalf'}, 'node_2': {'label': 'Person', 'name': 'Frodo'}, 'relationship': 'Gandalf suspects the Ring is a Ring of Power and counsels Frodo to take it away from the Shire.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Gandalf'}, 'node_2': {'label': 'Person', 'name': 'Sauron'}, 'relationship': 'Gandalf confirms the Ring was lost by the Dark Lord Sauron.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Gandalf'}, 'node_2': {'label': 'Person', 'name': 'Frodo'}, 'relationship': 'Gandalf counsels Frodo to take the Ring away from the Shire.', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Frodo'}, 'node_2': {'label': 'Person', 'name': 'Pippin Took'}, 'relationship': 'Frodo sets out on foot with his cousin Pippin 

In [8]:
from knowledge_graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()

aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called


20